Vaccination data is [here](https://covid.cdc.gov/covid-data-tracker/#vaccinations_vacc-total-admin-rate-total) and ICU data [here](https://coronavirus.jhu.edu/data/hospitalization-7-day-trend). The latter dataset is uploaded [here](https://github.com/RInterested/DATASETS/blob/gh-pages/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_State_Timeseries.csv). [This](https://stats.stackexchange.com/a/26779/67822) is what I intend to do.

In [7]:
install.packages('RCurl')
require(RCurl)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



Collecting the number of ICU dedicated to COVID confirmed or suspected cases per state as a measure of serious disease:

In [11]:
url <- "https://raw.githubusercontent.com/RInterested/DATASETS/gh-pages/COVID19%20Reported%20Patient%20Impact%20and%20Hospital%20Capacity%20by%20State%20Timeseries"
dat <- read.csv(url)
data <- dat[,c(1,2,27,39)]
colnames(data) <- c('state','date','covid','ICU.beds')
data[,2] <- as.Date(data$date)
df <- data[!(rowSums(is.na(data))),]
df$non.covid <- df[,4]-df[,3]
df <- df[,c('state','date','covid','non.covid','ICU.beds')]
df <- df[order(df[,'state']), ]
head(df)

,state,date,covid,non.covid,ICU.beds
,<chr>,<date>,<int>,<int>,<int>
73,AK,2020-07-29,11,32,43
381,AK,2020-08-29,9,37,46
393,AK,2020-12-20,20,106,126
409,AK,2020-12-04,33,98,131
459,AK,2021-01-10,11,107,118
681,AK,2020-12-30,11,116,127


Now the data in the last 30 days of delta wave in the US will be average state-wise. For example:

In [12]:
# Example Alabama:
n <- 30 # number of days to average
AL <- df[df$state=="AL",]
AL <- AL[order(as.Date(AL$date,format = "%d/%m/%Y")),]
row.names(AL) <- 1:nrow(AL)
round(colMeans(tail(AL[,c(3,4,5)],n)))

covid non.covid  ICU.beds 
      836       745      1581

Performing the same averaging for all states:

In [17]:
states <- unique(df[,1])
m <- matrix(0,length(states),3)

for(i in 1:length(states)){
  temp <- df[df$state==states[i],]
  temp <- temp[order(as.Date(temp$date,format = "%d/%m/%Y")),]
  m[i,1:3] <- round(colMeans(tail(temp[,c(3,4,5)],n)))
}
icu <- cbind.data.frame(states,m)
icu <- icu[order(icu[,'states']), ]
names(icu) <- c('states','covid','non.covid','ICU.beds')
icu$frac <- round(icu$covid / icu$ICU.beds, 2)
head(icu)

,states,covid,non.covid,ICU.beds,frac
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,AK,35,89,123,0.28
2,AL,836,745,1581,0.53
3,AR,457,547,1004,0.46
4,AS,0,7,7,0.00
5,AZ,478,1689,2167,0.22
6,CA,1935,5304,7240,0.27


Getting the vaccination data per state and tidying up the dataset:

In [19]:
y = url("https://raw.githubusercontent.com/RInterested/DATASETS/gh-pages/covid19_vaccinations_in_the_united_states.csv")
vaccines <- read.csv(y)
vaccines[1:3,1:3]
vac <- vaccines[,c(1,3)]
names(vac) <- c('states','doses.per.100k')

s = url("https://raw.githubusercontent.com/RInterested/DATASETS/gh-pages/states.csv")
state <- read.csv(s)

vac <- vac[vac[,1] %in% state[,1],]

for(i in 1:nrow(vac)){
    vac[i,1] <- state[which(state[,1]==vac[i,1]),2]
}

vac[,2] <- as.numeric(vac[,2])
head(vac)

,State.Territory.Federal.Entity,Total.Doses.Delivered,Doses.Delivered.per.100K
,<chr>,<int>,<chr>
1,Alaska,988195,135083
2,Alabama,6388850,130300
3,Arkansas,3851970,127641


,states,doses.per.100k
,<chr>,<dbl>
1,AK,135083
3,AR,127641
4,AS,111404
5,AZ,131204
7,CA,142990
8,CO,137594


Merging ICU occupancy and vaccination datasets:

In [20]:
d <- merge(icu,vac, by="states")
head(d)

,states,covid,non.covid,ICU.beds,frac,doses.per.100k
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,AK,35,89,123,0.28,135083
2,AR,457,547,1004,0.46,127641
3,AS,0,7,7,0.00,111404
4,AZ,478,1689,2167,0.22,131204
5,CA,1935,5304,7240,0.27,142990
6,CO,273,1079,1353,0.20,137594


Checking whether the logistic regression of covid cases in the ICU regressed over the vaccination status of the population in different states is significant:

In [24]:
fit <- glm(cbind(d$covid,d$non.covid) ~ scale(d$doses.per.100k), 
           family='binomial')

summary(fit)


Call:
glm(formula = cbind(d$covid, d$non.covid) ~ scale(d$doses.per.100k), 
    family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-27.566   -7.525   -2.758    2.623   39.715  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -0.871522   0.007831 -111.30   <2e-16 ***
scale(d$doses.per.100k) -0.193593   0.010832  -17.87   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 8346.2  on 51  degrees of freedom
Residual deviance: 8024.5  on 50  degrees of freedom
AIC: 8375.2

Number of Fisher Scoring iterations: 4


The scaling and centering is meant to estimate the effect size:

In [25]:
exp(cbind(OR = coef(fit), confint(fit)))

Waiting for profiling to be done...



,OR,2.5 %,97.5 %
(Intercept),0.4183142,0.4119365,0.4247772
scale(d$doses.per.100k),0.8239930,0.8066747,0.8416660


So a one-unit increase in the vaccination variable, decreases 20% the odds of being in the ICU.